# Detecting and Classifying Toxic Comments
# Part 3-1: TF*IDF & Random Forest Classifiers

It may be possible to employ sequential binary models in order to get better results with rarer cases.

If we first classify Toxic and Not Toxic, we could further process only the Toxic results against models that had been trained only to recognise sub-classes of toxic models.

## Python Library Imports

In [15]:
import pandas as pd
import numpy as np

## Import spaCy

In [16]:
import spacy

from spacy.lang.en import English
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

from spacy.tokens import Doc

# import custom trained spaCy model
nlp = spacy.load("../models/spacy_2/")

## Import nltk

In [17]:
# nltk imports
import nltk
from nltk.corpus import stopwords

## Import Custom Functions

In [18]:
import sys

# add src folder to path
sys.path.insert(1, '../src')

# from text_prep import tidy_series, uppercase_proportion_column
from spacy_helper import doc_check

# Getting info from preserved spaCy docs

I've had a little difficulty with getting the doc properties to un-pickle and maintain the ability to further process them later. docs seem to depend on some vocab properties of the model that are not saved within the doc itself.

In [5]:
%%time
'''
CPU times: user 3min 44s, sys: 33.5 s, total: 4min 17s
Wall time: 4min 32s
'''
X_train = pd.read_pickle('../data/basic_df_split/X_train_2-1.pkl')

CPU times: user 3min 39s, sys: 28.7 s, total: 4min 8s
Wall time: 4min 16s


In [6]:
# load y_train
! ls ../data/basic_df_split/
y_train = pd.read_pickle('../data/basic_df_split/basic_y_train.pkl')
X_test = pd.read_pickle('../data/basic_df_split/basic_X_test.pkl')
y_test = pd.read_pickle('../data/basic_df_split/basic_y_test.pkl')


X_train_2-1.pkl         basic_X_test.pkl        basic_y_test.pkl
X_train_docs_series.pkl basic_X_train.pkl       basic_y_train.pkl


In [7]:
X_train.columns

Index(['comment_text', 'uppercase_proportion', 'docs', 'lemmas', 'doc_vectors',
       'tok_vectors'],
      dtype='object')

### Create list of lemmas, less nltk stopwords

In [8]:
stopw_set = set(stopwords.words('english'))

In [9]:
%%time
# remove lemmas that appear in nltk stopword list
X_train['lemmas_less'] = X_train['lemmas'].apply(lambda row: [lemma for lemma in row if lemma not in stopw_set])

CPU times: user 1.48 s, sys: 993 ms, total: 2.47 s
Wall time: 3.03 s


## Further reduce lemmas by min_length & max_length

Exploration of corpus vocabulary suggests that lemmas of 2 or fewer characters are likely not very useful and can be removed to reduce features. 

Lemmas of longer than 20 characters are often run-on words (where spaces have been omitted). Although a few of them have words hidden within them that may be considered toxic, the rarity and non-standard format make them unlikely to be generalizable.

In [10]:
%%time
min_l = 3
max_l = 20

X_train['lemmas_less'] = X_train['lemmas'].apply(lambda row: [lemma for lemma in row if len(lemma) >= min_l or len(lemma) <= max_l])

CPU times: user 663 ms, sys: 77.1 ms, total: 740 ms
Wall time: 817 ms


# TF*IDF

## Scikit Learn Imports

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
%%time
tfidf_sklearn = TfidfVectorizer(ngram_range = (1,3),
                                min_df = 2)

# return sparse matrix
# join list into individual strings
tfidf_values = tfidf_sklearn.fit_transform(X_train['lemmas_less'].apply(lambda x: " ".join(x)))

CPU times: user 17.1 s, sys: 1.19 s, total: 18.3 s
Wall time: 18.5 s


In [13]:
'''
last values (with min_df as 1):
<106912x4089577 sparse matrix of type '<class 'numpy.float64'>'
	with 8002628 stored elements in Compressed Sparse Row format>
'''

tfidf_values

<106912x459917 sparse matrix of type '<class 'numpy.float64'>'
	with 4387497 stored elements in Compressed Sparse Row format>

In [14]:
%%time
'''
CPU times: user 11.9 s, sys: 8.27 s, total: 20.2 s
Wall time: 23 s
'''
lemmas_less_tfidf = pd.DataFrame(tfidf_values.toarray(),
                                 columns=tfidf_sklearn.get_feature_names())

CPU times: user 3.86 s, sys: 4.86 s, total: 8.72 s
Wall time: 9.76 s


In [15]:
lemmas_less_tfidf.shape

(106912, 459917)

In [16]:
y_train['toxic'].shape

(106912,)

In [17]:
# sum(lemmas_less_tfidf['jerk'])

# search_term = 'jerk'

# # running this portion will crash the kernel
# bool_mask = lemmas_less_tfidf.sort_values(search_term, ascending=False)[search_term][:10]
# bool_mask

# Toxic: Random Forest Classifier

Resources:
- [Explanation of Warm Start for RFC (not what you may think)](https://stackoverflow.com/questions/42757892/how-to-use-warm-start/42763502)  
- [TfidfVectorizer Docs](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)  

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [19]:
toxic_rfc = RandomForestClassifier(n_estimators=100,
                                   max_depth = 10,
                                   oob_score=True,
                                   n_jobs=-1,
                                   random_state=42,
                                   warm_start=True)

In [20]:
X = lemmas_less_tfidf
y = y_train

In [21]:
toxic_logistic = LogisticRegression(warm_start=True,
                                    random_state=42,
                                    verbose=True,
                                    solver='sag',
                                    multi_class='ovr',
                                    max_iter=100,
                                    n_jobs=-1)

In [79]:
y.columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [80]:
%%time
toxic_logistic.fit(X[:10],y['obscene'][:10])

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [ ]:
def batch_train(X, y, model, batch_size=1000, verbose=False, start=0):
    
    remaining = len(X)
    
    if batch_size > remaining:
        bach_size = remaining
    
    b = start
    e = batch_size
    
    while e <= remaining:
        
        model.fit(X[b:e], y[b:e])
        
        
        
        
        
    
    
    
    
    

In [21]:
# 

In [22]:
# naive_toxic = BernoulliNB
# print(vect_X_train.shape)
# print(y_train.shape)
# print(X_train.shape)

# vect_X_train.iloc[0]

# naive_toxic.fit(vect_X_train, y_train)

In [19]:
%load_ext autoreload
%autoreload 2

import sys

# add src folder to path
sys.path.insert(1, '../tokens')

# from text_prep import tidy_series, uppercase_proportion_column
from twitter_tok import twitter_api_key, api_secret_key, bearer_token, eml, password

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import requests

In [4]:
# import sys

# class ListStream:
#     def __init__(self):
#         self.data = []
#     def write(self, s):
#         self.data.append(s)

# sys.stdout = x = ListStream()

# for i in range(2):
#     print ('i = ', i)

# sys.stdout = sys.__stdout__
# print(x.data)

https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/master/Sampled-Stream/sampled-stream.py

https://developer.twitter.com/en/docs/twitter-api/tweets/sampled-stream/introduction

In [21]:
%load_ext autoreload
%autoreload 2

sys.path.insert(1, '../tokens')
from twitter_stream import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import requests
import os
import json


In [1]:
import os 
import tweepy as tw
import pandas as pd

Had little luck with get old tweets

https://pypi.org/project/GetOldTweets3/



In [28]:
import twitter_rules

In [63]:
# twitter_rules.main()
list_ = list()



twitter_rules.main(text_list=list_, total_tweets=10)

{"data": [{"id": "1377745160801562632", "value": "-is:retweet (place_country:US) lang:en"}], "meta": {"sent": "2021-04-01T22:28:52.615Z"}}
{"meta": {"sent": "2021-04-01T22:28:53.935Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "-is:retweet (place_country:US) lang:en", "id": "1377749842391351298"}], "meta": {"sent": "2021-04-01T22:28:55.380Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200


In [65]:
print(len(list_))
list_

10


['"There are, essentially, two Californias:"',
 '"@tesspro @StephenKing @GovRonDeSantis By the way, South Dakota is awesome. I\\u2019m not sure why you would, but it seems you may have been mocking where I live. Says the guy from the Inited States..."',
 '"It\\u2019s beginning"',
 '"Oh, yeah. This is why I shouldn\\u2019t watch so much Minnesota sports. It makes me feel bad. A loss stolen from the jaws of victory! #MNTwins"',
 '"@RichieWalsh 7 pitchers in one 9 inning game? Would\'ve been 8 pitchers if Vasquez was eligible for work release."',
 '"CNBC: Biden asks Education secretary to see if he can legally cancel student debt.\\n\\nC\'mon man, let\'s go! \\n\\nhttps://t.co/w5A9bkotqh\\n\\nvia @GoogleNews"',
 '"I would appreciate if everyone stopped attacking me for a tweet that was in the past"',
 '"And what https://t.co/kzpmaU5atl"',
 '"I could straight up murder a snowcone and nachos rn"',
 '"\\u201cThe crooks are going to get a gun!\\u201d said the guy who is okay making it a crime

Resources:
- [Building Twitter Rules](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/integrate/build-a-rule)  